#### -- [INIT] -- 
***

##### **Import Libraries and Init Globals**
- `on_awake   ` (init_game => screen, refresh, load assets ... etc)
- `on_exit    ` (close_app => clear memory and shut down cpu/sys ... etc)

In [1]:
import pygame
from pygame.event import Event
from pygame.math import Vector2
from typing import Tuple, List, Callable, Optional, Type
import logging
log_level = logging.DEBUG
logging.basicConfig(level=log_level)

# check environment variables
import os
from dotenv import load_dotenv
load_dotenv() # This loads the environment variables from the .env file
# Now you can access the environment variables using os.getenv('variable_name')
production = os.getenv('PRODUCTION')
logging.debug(f'PRODUCTION={production}')

# imports context from library.system_context module
from library.pygame_system_context import PygameSystemContext as context

DEBUG:root:PRODUCTION=1


pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### -- [ENTITY] --
***
- generate a `Unique ID` for each `entity`

In [2]:
from library.entity import Entity

#### -- [COMPONENT] --
***

##### **Component Base**
These classes should ONLY contain DATA => NO FUNCTIONS
- `Display`     : Position, Shape
- `Interaction` : Collider, Health

In [3]:
from library.component import ComponentABC, ABC, abstractmethod
from collections import namedtuple


# common data types
KeyFrame = namedtuple("KeyFrame", ["tick", "value"])


# input handler
class Input(ComponentABC):
    def __init__(self):
        pass


# entity category
class Player(ComponentABC):
    def __init__(self):
        pass


class Asteroid(ComponentABC):
    def __init__(self):
        pass


class HasKeyFrame(ComponentABC):
    def __init__(self, key_frame: KeyFrame):
        """
         KeyFrame
        - tick  == current angle
        - value == rotation speed
        """
        self.key_frame = key_frame


# transforms
class Transform(ComponentABC):
    def __init__(
        self,
        center: Vector2 = Vector2(0, 0),
        # default forward direction is up ^
        rotation: float = 0,
        scale: float = 1.0,
    ):
        """transform container for center and forward direction"""
        self.center = center
        self.rotation = rotation
        self.scale = scale


class Position(ComponentABC):
    def __init__(self, position: Vector2 = Vector2(0, 0)):
        self.position = position


class Rotation(ComponentABC):
    def __init__(self, angle: float):
        self.angle = angle


class Velocity(ComponentABC):
    def __init__(self, vx: float, vy: float, magnitude: float = 1):
        """velocity == normalized input(vx,vy) * magnitude"""
        self.vx = vx
        self.vy = vy
        self.magnitude = magnitude


# state
class Health(ComponentABC):
    def __init__(self, health: int):
        self.health = health


# collision volume
class Collider(ComponentABC):
    def __init__(self, radius: float):
        self.radius = radius


# graphics
class Shape(ComponentABC):
    def __init__(self, vertices: List[List[Vector2]]):
        self.vertices_source = vertices
        # use this to transform source points for rendering
        self.vertices_render = vertices


##### **PubSub Manager**
- `subscribe` to `event` and pass `callback`
- `publish` event

In [4]:
"""
If `pubsub_event_manager.py` is saved under a folder named `library`, you can 
import `PubSubEventManager` using a slightly modified import statement in your 
other Python files:
- First, make sure that the `library` folder contains an empty file named `__init__.py`. 
- This file tells Python that the folder should be treated as a packa∏ge.
"""

from library.pubsub_event_manager import PubSubEventManager

##### **Component Manager**
- `add`
- `remove`
- `get`
- `gather` entities `given` component set

In [5]:
from library.ecs_manager import ComponentManager

#### -- [ASSETS] --

##### **Vector2 Transform**

In [6]:
from library.vector2transform import Vector2Transform

##### **Vertex Assets**
- List of x,y `points`

In [7]:
import math
from numpy import random

class VertexAsset:

    @staticmethod
    def _generate_circle(
        radius: float, num_points: int, func: Callable[[float], float] = lambda x: x
    ) -> List[Vector2]:
        """return a list of points that form a circle"""
        points : List[Vector2] = []
        angle = 2 * math.pi / num_points

        for i in range(num_points):
            # length = random.randint(int(radius * 0.5), radius)
            length = func(radius)
            x = length * math.cos(angle * i)
            y = length * math.sin(angle * i)
            points.append(Vector2(x, y))

        return points

    @staticmethod
    def generate_asteroid(radius, num_points) -> List[Vector2]:
        """return a list of points that form an asteroid by adding noise to radius"""
        # `numpy.random.uniform` function to generate noise with a more uniform
        # distribution between the minimum and maximum radii
        noise_lambda = lambda x: random.uniform(x * 0.5, x)
        points = VertexAsset._generate_circle(radius, num_points, noise_lambda)
        return points

    @staticmethod
    def generate_ship(radius) -> List[Vector2]:
        """return a list of points that form a ship"""
        points = VertexAsset._generate_circle(radius, 3)
        return points


##### **Spawn Game Objects**

In [8]:
from typing import NamedTuple


class ShipConfig(NamedTuple):
    HEALTH = 1
    SIZE = 30


class AsteroidConfig(NamedTuple):
    HEALTH = 1
    SIZE = 100
    VERTICES = 10
    K_POSITION = 2  # change in position per tick
    K_ROTATION = 2  # change in rotation per tick


class O_Type:
    """ 
    Object Configurations for all game objects to be spawned 
    """
    SHIP = ShipConfig()
    ASTEROID = AsteroidConfig()


class O_Factory:
    """
    Object Factory to instance and spawn game objects
    """

    @staticmethod
    def spawn_ship(component_manager: ComponentManager) -> None:
        "spawn a ship at the center of the screen"
        ship = Entity(name=f"ship_{component_manager.entities_count}")
        ship_game_object = Player()
        ship_input = Input()
        ship_transform = Transform()
        ship_position = Position(
            Vector2(
                int(context.CONFIG.WIDTH * 0.5),
                int(context.CONFIG.HEIGHT * 0.5),
            )
        )
        ship_rotation = Rotation(0)
        ship_velocity = Velocity(0, 0, 0)
        ship_health = Health(O_Type.SHIP.HEALTH)
        ship_points = VertexAsset.generate_ship(O_Type.SHIP.SIZE)
        ship_shape = Shape([ship_points])

        component_manager.add_component(ship, ship_game_object)
        component_manager.add_component(ship, ship_input)
        component_manager.add_component(ship, ship_transform)
        component_manager.add_component(ship, ship_position)
        component_manager.add_component(ship, ship_rotation)
        component_manager.add_component(ship, ship_velocity)
        component_manager.add_component(ship, ship_health)
        component_manager.add_component(ship, ship_shape)

    @staticmethod
    def spawn_asteroid(component_manager: ComponentManager) -> None:
        "spawn an asteroid at a random positions"
        asteroid = Entity(name=f"asteroid_{component_manager.entities_count}")
        asteroid_game_object = Asteroid()
        angle_degree = random.randint(0, 359)
        rotation_speed = random.randint(
            -O_Type.ASTEROID.K_ROTATION, O_Type.ASTEROID.K_ROTATION
        )
        asteroid_has_key_frame = HasKeyFrame(KeyFrame(angle_degree, rotation_speed))
        asteroid_transform = Transform()
        asteroid_position = Position(
            Vector2(
                random.randint(0, context.CONFIG.WIDTH),
                random.randint(0, context.CONFIG.HEIGHT),
            )
        )
        asteroid_rotation = Rotation(random.randint(0, 359))
        asteroid_velocity = Velocity(
            random.randint(
                -O_Type.ASTEROID.K_POSITION, O_Type.ASTEROID.K_POSITION
            ),
            random.randint(
                -O_Type.ASTEROID.K_POSITION, O_Type.ASTEROID.K_POSITION
            ),
            1,
        )
        asteroid_health = Health(O_Type.ASTEROID.HEALTH)
        asteroid_points = VertexAsset.generate_asteroid(
            random.randint(int(O_Type.ASTEROID.SIZE * 0.5), O_Type.ASTEROID.SIZE),
            O_Type.ASTEROID.VERTICES,
        )
        asteroid_shape = Shape([asteroid_points])

        component_manager.add_component(asteroid, asteroid_game_object)
        component_manager.add_component(asteroid, asteroid_has_key_frame)
        component_manager.add_component(asteroid, asteroid_transform)
        component_manager.add_component(asteroid, asteroid_position)
        component_manager.add_component(asteroid, asteroid_rotation)
        component_manager.add_component(asteroid, asteroid_velocity)
        component_manager.add_component(asteroid, asteroid_health)
        component_manager.add_component(asteroid, asteroid_shape)

    @staticmethod
    def spawn_asteroids(num_asteroids, component_manager: ComponentManager) -> None:
        "spawn a number of asteroids at random positions"
        for i in range(num_asteroids):
            O_Factory.spawn_asteroid(component_manager)


#### -- [SYSTEM]--
***

In [9]:
class FrameUpdateSystemABC(ABC):
    """Interface for Frame Update System with cached screen context"""

    def __init__(self, screen: pygame.Surface) -> None:
        self.screen = screen

    @abstractmethod
    def update(self, **kwargs) -> None:
        pass


class EntitySystemABC(ABC):
    """Interface for Entity System that updates entities cached from ComponentManager"""

    def __init__(
        self,
        c_manager: ComponentManager,
        event_manager: PubSubEventManager,
        components: Optional[List[Type[ComponentABC]]] = None,
    ) -> None:
        """
        Initialize the SystemABC instance
        :param c_manager: ComponentManager events subscribe to change and regather cached entities
        :param components: List of components to filter entities
        """
        self._c_manager = c_manager
        self.event_manager = event_manager
        # @todo : Subscribe to events
        event_manager.subscribe(
            ComponentManager.ADD_COMPONENT, self.on_add_component_dirty
        )
        event_manager.subscribe(
            ComponentManager.REMOVE_COMPONENT, self.on_remove_component_dirty
        )

        self._components = components
        self.entities: Optional[List[Entity]] = None
        self.entities = EntitySystemABC.gather_system_entities(
            self._c_manager, self._components
        )
        print(
            f"[entities] on_init gathering system cache for {self.__class__.__name__}"
        )

    def on_add_component_dirty(self, entity, component) -> None:
        self.set_dirty()

    def on_remove_component_dirty(self, entity, component_type) -> None:
        print(f"e[entities] on_remove_component_dirty {entity} {component_type}")
        self.set_dirty()

    def update(self, **kwargs) -> None:
        """Update the system with the given component manager"""
        pass

    def set_dirty(self) -> None:
        print(
            f"[entities] on_dirty regathering sytem cache for {self.__class__.__name__}"
        )
        self.entities = EntitySystemABC.gather_system_entities(
            self._c_manager, self._components
        )

    @staticmethod
    def gather_system_entities(
        c_manager: ComponentManager, components: Optional[List[Type[ComponentABC]]]
    ) -> Optional[List[Entity]]:
        """return a list of entities that have all the given components"""
        # Regather entities if not intialized or if the component manager is dirty
        # @todo : also check is_dirty flag
        entities = None
        if components is None:
            return entities
        else:
            entities = c_manager.gather_entities_with_all_components(components)
        return entities


##### **Handle Inputs**

In [10]:
class InputSystem(EntitySystemABC):
    def _on_keydown(self, entity, event) -> None:
        if event.key == pygame.K_UP:
            velocity = self._c_manager.get_component(entity, Velocity)
            velocity.magnitude += 1
        elif event.key == pygame.K_DOWN:
            velocity = self._c_manager.get_component(entity, Velocity)
            velocity.magnitude -= 1
        if event.key == pygame.K_RIGHT:
            rotation = self._c_manager.get_component(entity, Rotation)
            rotation.angle = (rotation.angle + 5) % 360
        elif event.key == pygame.K_LEFT:
            rotation = self._c_manager.get_component(entity, Rotation)
            rotation.angle = (rotation.angle - 5) % 360

    def _on_keyup(self, entity, event) -> None:
        velocity = self._c_manager.get_component(entity, Velocity)

    def update(self, event: pygame.event.Event, **kwargs) -> None:
        """Handle input events for entities with an input component"""
        super().update(**kwargs)
        # early exit if no event
        if event is None:
            return
        # early exit if no entities
        if self.entities is None:
            return
        for entity in self.entities:
            if event.type == pygame.KEYDOWN:
                self._on_keydown(entity, event)
            elif event.type == pygame.KEYUP:
                self._on_keyup(entity, event)


##### **Position and Rotate**

In [11]:
class PositioningSystem(EntitySystemABC):
    @staticmethod
    def _screen_wrap(position: Vector2) -> Vector2:
        """Wrap a position to the screen"""
        return Vector2(
            position.x % context.CONFIG.WIDTH, position.y % context.CONFIG.HEIGHT
        )

    @staticmethod
    def _forward_position(
        position: Vector2, angle_degrees: float, magnitude: float
    ) -> Vector2:
        """Calculate the forward position of an entity"""
        # Convert angle from degrees to radians
        angle_radians = math.radians(angle_degrees)

        # Calculate the change in x and y in the forward direction
        dx = magnitude * math.cos(angle_radians)
        dy = magnitude * math.sin(angle_radians)

        # Calculate the new position given radian and magnitude
        dir_x = position.x + dx
        dir_y = position.y + dy

        return Vector2(dir_x, dir_y)

    def update(self, **kwargs) -> None:
        """
        Update the position of entities with a velocity component
        - with screen wrapping
        """
        super().update(**kwargs)
        if self.entities is None:
            return
        for entity in self.entities:
            p = self._c_manager.get_component(entity, Position)
            magnitude = self._c_manager.get_component(entity, Velocity).magnitude
            # Update position based on velocity and rotation if necessary
            angle = self._c_manager.get_component(entity, Rotation).angle
            # calculate forward position
            forward_position = PositioningSystem._forward_position(
                p.position, angle, magnitude
            )
            # update to forward position (with screen wrapping)
            p.position = PositioningSystem._screen_wrap(forward_position)


class RotationalSystem(EntitySystemABC):
    def update(self, **kwargs) -> None:
        """Rotate entities with a rotation component"""
        super().update(**kwargs)
        if self.entities is None:
            return
        for entity in self.entities:
            rotation = self._c_manager.get_component(entity, Rotation)
            shape = self._c_manager.get_component(entity, Shape)
            rotated_meshes: List[List[Vector2]] = []
            for mesh in shape.vertices_source:
                rotated_mesh = Vector2Transform.rotate_vectors(mesh, rotation.angle)
                rotated_meshes.append(rotated_mesh)
            shape.vertices_render = rotated_meshes


##### **Apply Animations**

In [12]:
class AnimateAsteroidsSystem(EntitySystemABC):
    def update(self, **kwargs) -> None:
        """Animate asteroids"""
        super().update(**kwargs)
        if self.entities is None:
            return
        for entity in self.entities:
            xform = self._c_manager.get_component(entity, Transform)
            animation = self._c_manager.get_component(entity, HasKeyFrame)
            # tick == current angle
            # value == angle increment
            # modulus 360 to keep angle in range
            angle_increment = animation.key_frame.value
            update_angle = (animation.key_frame.tick + angle_increment) % 360
            animation.key_frame = KeyFrame(update_angle, angle_increment)
            xform.rotation = update_angle


##### **Draw Shapes**

In [13]:
class RenderShape:
    @staticmethod
    def draw(screen: pygame.Surface, cm: ComponentManager, entity: Entity) -> None:
        """Draw a shape to the screen"""
        position = cm.get_component(entity, Position).position
        shape = cm.get_component(entity, Shape)
        # audit 🛑 : check if most optimized way to draw a collection of shape
        for mesh in shape.vertices_render:
            transformed_vertices = [position + vertex for vertex in mesh]
            pygame.draw.polygon(
                screen,
                (255, 255, 255),
                transformed_vertices,
                1,
            )


class ShapeDrawSystem(EntitySystemABC):
    def __init__(
        self,
        c_manager: ComponentManager,
        event_manager : PubSubEventManager,
        components: List[Type[ComponentABC]],
        screen: pygame.Surface,
        clock: pygame.time.Clock,
    ):
        """Render to surface with given clock"""
        super().__init__(c_manager, event_manager, components)
        self.screen = screen
        self.clock = clock

    def update(self, **kwargs) -> None:
        super().update(**kwargs) 
        # Render all entities with a position and points component
        if self.entities is None:
            return
        for entity in self.entities:
            RenderShape.draw(self.screen, self._c_manager, entity)


##### **UI and Text Blit**

In [14]:
class TextBlitSystem(FrameUpdateSystemABC):
    def __init__(
        self,
        screen: pygame.Surface,
        position: Tuple[int, int],
        font_size=24,
        color=(255, 255, 255),
    ):
        """Draw text to surface at given position"""
        super().__init__(screen)
        self.position = position
        self.font = pygame.font.SysFont("none", font_size)
        self.color = color

    def update(self, message: str, **kwargs) -> None:
        super().update(**kwargs)
        text_surface = self.font.render(message, True, self.color)
        self.screen.blit(text_surface, self.position)


##### **End Frame Tick**

In [15]:
class EndFrameTickSystem(FrameUpdateSystemABC):
    def __init__(self, screen: pygame.Surface, clock: pygame.time.Clock):
        """End frame and tick clock"""
        super().__init__(screen)
        self.screen = screen
        self.clock = clock

    def update(self, **kwargs) -> None:
        super().update(**kwargs)
        self.screen.fill(context.CONFIG.BACKGROUND_COLOR)
        self.clock.tick(context.CONFIG.FPS)


#### -- [RUNTIME] --
***

##### **Debug Log**

In [16]:
def debug_player_rotation(c_manager: ComponentManager) -> Optional[Rotation]:
    """Debug player rotation"""
    player_entities = c_manager.gather_entities_with_all_components(
        [Shape, Input, Rotation]
    )
    if player_entities:
        player_rotation = c_manager.get_component(player_entities[0], Rotation)
    else:
        player_rotation = None

    return player_rotation

##### **Handle Events**

In [17]:
def handle_events(
    frame_events: List[Event],
    input_system: InputSystem,
    is_super_hot: bool,
    c_manager: ComponentManager,
) -> bool:
    for event in frame_events:
        if event.type == pygame.QUIT:
            context.on_exit()
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_ESCAPE:
                context.on_exit()
            elif event.key == pygame.K_SPACE:
                is_super_hot = not is_super_hot
            elif event.key == pygame.K_j:
                # delete an asteroid
                entities = c_manager.gather_entities_with_all_components([Asteroid, Shape])
                if entities:
                    print(f"Deleting Asteroid {entities[0]}")
                    c_manager.remove_component(entities[0], Shape)
        # check if the event a keyboard or mouse event
        if event.type in (
            pygame.KEYDOWN,
            pygame.KEYUP,
            pygame.MOUSEBUTTONDOWN,
            pygame.MOUSEBUTTONUP,  # pygame.MOUSEMOTION
        ):
            # Update input system
            input_system.update(event=event)
    return is_super_hot


##### **Main**

In [18]:
def main() -> None:
    # [INIT] - systems, context and COMPONENT managers
    context_sys = context.on_awake()
    event_manager = PubSubEventManager()
    c_manager = ComponentManager(event_manager)

    # [INIT] - game ENTITIES
    # @note 🧠 : Populate entities BEFORE systems, so we can cache entities on
    # system inits
    # Create ship
    O_Factory.spawn_ship(c_manager)
    # Create asteroid
    O_Factory.spawn_asteroids(5, c_manager)

    # [INIT] - game SYSTEMS
    input_system = InputSystem(c_manager, event_manager, [Input, Velocity, Rotation])
    positioning_system = PositioningSystem(
        c_manager, event_manager, [Shape, Position, Rotation, Velocity]
    )
    rotational_system = RotationalSystem(c_manager, event_manager, [Shape, Rotation])
    animate_asteroids_system = AnimateAsteroidsSystem(
        c_manager, event_manager, [Asteroid, HasKeyFrame, Transform]
    )
    # get entities with POSITION (where) AND SHAPE (what) to render
    shape_draw_system = ShapeDrawSystem(
        c_manager,
        event_manager,
        [Shape, Position],
        context_sys.screen,
        context_sys.clock,
    )

    # [INIT] - frame SYSTEMS
    text_blit_system_fps = TextBlitSystem(context_sys.screen, (280, 430))
    text_blit_system_event = TextBlitSystem(context_sys.screen, (10, 450), 18)
    end_frame_tick_system = EndFrameTickSystem(context_sys.screen, context_sys.clock)

    """DEBUG"""
    is_super_hot = False
    text_blit_system_debug_player_rotation = TextBlitSystem(
        context_sys.screen, (280, 410)
    )
    debug_rotation = debug_player_rotation(c_manager)
    """DEBUG"""

    # Main game loop
    while True:
        try:
            # Event Handling
            frame_events = pygame.event.get()
            if is_super_hot:
                # if is_super_hot == True, only update tick if on frame event
                if len(frame_events) == 0:
                    continue
            is_super_hot = handle_events(
                frame_events, input_system, is_super_hot, c_manager
            )
            text_blit_system_event.update(message=f"Event: {frame_events}")
            # Update positions
            positioning_system.update()
            # Update rotation
            rotational_system.update()
            # Apply animations
            animate_asteroids_system.update()
            # Draw shapes
            shape_draw_system.update()
            # Blit text
            text_blit_system_fps.update(
                message=f"FPS: {context_sys.clock.get_fps():.2f}"
            )
            if debug_rotation is not None:
                text_blit_system_debug_player_rotation.update(
                    message=f"Player rotation: {debug_rotation.angle:.2f}",
                )
            # Update display
            pygame.display.update()
            # End frame tick
            end_frame_tick_system.update()

        except SystemExit:
            break


# Only run if this is being executed directly
# - prevents the code from being run when imported
# @note 🧠 : __name__ is built-in variable in Python that represents :
# - the name of the current module
# - When a script is run, Python sets the `__name__` variable to `'__main__'`
# for the script being executed
if __name__ == "__main__":
    main()


[entities_debug_print]: [1]/[6] with components [3] ['Input', 'Velocity', 'Rotation'] gathered [1] ['ship']
[entities] on_init gathering system cache for InputSystem
[entities_debug_print]: [6]/[6] with components [4] ['Shape', 'Position', 'Rotation', 'Velocity'] gathered [6] ['ship', 'asteroid', 'asteroid', 'asteroid', 'asteroid', 'asteroid']
[entities] on_init gathering system cache for PositioningSystem
[entities_debug_print]: [6]/[6] with components [2] ['Shape', 'Rotation'] gathered [6] ['ship', 'asteroid', 'asteroid', 'asteroid', 'asteroid', 'asteroid']
[entities] on_init gathering system cache for RotationalSystem
[entities_debug_print]: [5]/[6] with components [3] ['Asteroid', 'HasKeyFrame', 'Transform'] gathered [5] ['asteroid', 'asteroid', 'asteroid', 'asteroid', 'asteroid']
[entities] on_init gathering system cache for AnimateAsteroidsSystem
[entities_debug_print]: [6]/[6] with components [2] ['Shape', 'Position'] gathered [6] ['ship', 'asteroid', 'asteroid', 'asteroid', 'as

#### -- [FOOTNOTES] --
***

In [23]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


Set env variable for python
***
In Visual Studio Code (VSCode) with Jupyter notebooks, you can set an environment  
variable for your Python kernel by creating a `.env` file in the same directory  
as your Jupyter notebook file.

Here's how to do it:

1. In the same directory as your Jupyter notebook file, create a new file named  
`.env`.

2. Open the `.env` file and add your environment variables in the format  
`KEY=VALUE`. For example, to set the `PRODUCTION` environment variable, add the  
following line:

```
PRODUCTION=1
```

3. Save the `.env` file.

4. Install the Python `python-dotenv` package if you haven't already. This  
package allows you to easily load environment variables from `.env` files. You  
can install it using pip:

```bash
pip install python-dotenv
```

5. In your Jupyter notebook, you can now load the environment variables from the  
`.env` file using the following code:

```python
import os
from dotenv import load_dotenv

load_dotenv()  # This loads the environment variables from the .env file

# Now you can access the environment variable using os.environ
production = os.environ.get("PRODUCTION")
print(production)
```

After following these steps, the environment variable `PRODUCTION` will be  
available in your Jupyter notebook running in VSCode.

Please note that you'll need to restart the Python kernel in your Jupyter  
notebook after creating or modifying the `.env` file to apply the changes.  
You can do this by clicking the "Restart" button (circular arrow) in the  
notebook toolbar.

In [19]:
%pip show rx

Name: Rx
Version: 3.2.0
Summary: Reactive Extensions (Rx) for Python
Home-page: http://reactivex.io
Author: Dag Brattli
Author-email: dag@brattli.net
License: MIT License
Location: /Users/mton/miniconda3/envs/torch-gpu/lib/python3.9/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install rx

Note: you may need to restart the kernel to use updated packages.


Reactive Python Examples
***

Python has a ReactiveX (Rx) library called `RxPY`. ReactiveX is a cross-platform  
library for programming with asynchronous data streams and reactive programming.  
RxPY is an implementation of the ReactiveX API for Python. It allows you to  
build event-driven and asynchronous applications using observable sequences and  
LINQ-style query operators.

To use RxPY, you need to install it first. You can do this using pip:

```bash
pip install rx
```

Once installed, you can use the library in your Python code. Here is a basic example:

```python
import rx
from rx import operators as ops

# Create an observable sequence
observable = rx.from_iterable([1, 2, 3, 4, 5])

# Apply a filter and a map operator to the sequence
filtered_mapped = observable.pipe(
    ops.filter(lambda x: x % 2 == 0),
    ops.map(lambda x: x * 2)
)

# Subscribe to the observable and print the results
filtered_mapped.subscribe(lambda x: print(f"Received: {x}"))
```

In this example, we create an observable sequence from an iterable  
(a list of integers). We then apply a filter to keep only even numbers and a  
map to double the values. Finally, we subscribe to the observable and print the  
received values.

You can find more information and examples in the :  
- [RxPY GitHub repository](https://github.com/ReactiveX/RxPY)  
- [official ReactiveX documentation](http://reactivex.io/intro.html).

Reactive Programming Notes
***
Reactive programming in Python, using libraries like RxPY, can be very useful  
for a variety of use cases. It provides an elegant and efficient way to handle  
asynchronous and event-driven programming. Some common use cases include:

1. **User interfaces**: Reactive programming can be used to manage user input  
events, GUI updates, and animations. By treating user actions as asynchronous  
event streams, you can easily compose and transform input events to update UI  
elements or trigger actions in response.

2. **Websockets and real-time applications**: Reactive programming is  
well-suited for handling real-time data communication over websockets.  
It can be used to manage data streams from multiple sources, perform  
transformations on the data, and respond to events as they occur.

3. **Data processing pipelines**: RxPY can be used to create data processing  
pipelines where data is ingested from various sources, transformed, and consumed  
by multiple observers. Reactive programming makes it easy to handle complex data  
flows, error handling, and resource management.

4. **Networking and distributed systems**: Reactive programming can manage  
asynchronous communication between distributed components or microservices.  
It helps to handle backpressure, retries, and timeouts in a clean and  
efficient manner.

5. **IoT and sensor data**: Reactive programming can be used to process and  
analyze data streams from IoT devices and sensors. It can be used to filter,  
aggregate, and transform data in real-time, and to react to specific events or  
conditions.

6. **APIs and web services**: Reactive programming can help manage API calls and  
web service requests in a non-blocking manner. You can use it to handle retries,  
timeouts, and error handling when making requests to external services.

7. **Concurrent and parallel programming**: Reactive programming can simplify  
the process of managing concurrent and parallel tasks, allowing you to  
coordinate and synchronize multiple asynchronous operations by using observable  
sequences.

These are just a few examples of where reactive programming in Python can be  
beneficial. The key advantage of using reactive programming is its ability to  
handle complex, asynchronous, and event-driven scenarios with ease, leading to  
more readable and maintainable code.

In an **`entity component system`**, where system caches entities on init suggest  
approaches to handle when the system **`cache is dirty`**
***
In an Entity Component System (ECS) architecture, systems cache entities during  
initialization, and sometimes the cache may become "dirty" when the state of  
entities or components changes during runtime. To handle such cases, you can use  
one of the following approaches:

1. **Flag-based approach**: Introduce a flag in the system that marks the cache  
as dirty. Whenever an entity or component is added, removed, or updated, set the  
dirty flag. In the system's update loop, check if the cache is dirty, and if so,  
update the cache before processing the entities.

```python
class ExampleSystem(EntitySystemABC):
    def __init__(self, c_manager):
        super().__init__(c_manager)
        self.cache_dirty = True
        self.entity_cache = []

    def update(self):
        if self.cache_dirty:
            self.update_cache()
            self.cache_dirty = False

        # process entities in cache

    def update_cache(self):
        # update entity_cache with the latest data
        pass

    def on_entity_added(self, entity):
        self.cache_dirty = True

    def on_entity_removed(self, entity):
        self.cache_dirty = True

    def on_component_updated(self, entity, component):
        self.cache_dirty = True
```

2. **Observer pattern**: Implement an observer pattern where systems subscribe  
to specific events related to entity and component changes. When such an event  
occurs, the corresponding system updates its cache accordingly.

```python
class ExampleSystem(EntitySystemABC):
    def __init__(self, c_manager):
        super().__init__(c_manager)
        self.entity_cache = []

        # Subscribe to events
        c_manager.subscribe("entity_added", self.on_entity_added)
        c_manager.subscribe("entity_removed", self.on_entity_removed)
        c_manager.subscribe("component_updated", self.on_component_updated)

    def update(self):
        # process entities in cache

    def on_entity_added(self, entity):
        # update entity_cache with the new entity
        pass

    def on_entity_removed(self, entity):
        # remove the entity from entity_cache
        pass

    def on_component_updated(self, entity, component):
        # update the entity_cache if necessary
        pass
```

3. **Partial cache updates**: Instead of updating the entire cache, track added,  
removed, or updated entities and components separately. Update the cache only  
for the affected entities when processing. This approach is more efficient for  
systems that handle a large number of entities.

```python
class ExampleSystem(EntitySystemABC):
    def __init__(self, c_manager):
        super().__init__(c_manager)
        self.entity_cache = []
        self.added_entities = set()
        self.removed_entities = set()
        self.updated_entities = set()

    def update(self):
        # process added, removed, and updated entities
        self.process_added_entities()
        self.process_removed_entities()
        self.process_updated_entities()

        # process remaining entities in cache

    def process_added_entities(self):
        # Process added entities and update entity_cache
        pass

    def process_removed_entities(self):
        # Process removed entities and update entity_cache
        pass

    def process_updated_entities(self):
        # Process updated entities and update entity_cache if necessary
        pass

    def on_entity_added(self, entity):
        self.added_entities.add(entity)

    def on_entity_removed(self, entity):
        self.removed_entities.add(entity)

    def on_component_updated(self, entity, component):
        self.updated_entities.add(entity)
```

Choose the approach that best fits your use case and performance requirements.

Explain the difference between `radian` and `angle` as if I am 5
***

Alright, let's use a simple example to explain the difference between an angle  
and a radian.

Imagine you have a yummy pizza. You want to cut the pizza into equal pieces. To  
do that, you need to cut it at a certain angle. An angle is a measure of how  
much you need to turn to go from one side of the pizza slice to the other. You  
can think of an `angle like a corner in a shape`.

Now let's talk about radians. Radians are just another way to measure angles.  
Imagine you have a piece of string that is the same length as the radius of the  
pizza (the distance from the center of the pizza to the edge). If you lay the  
string along the edge of a pizza slice and it fits perfectly from one side of  
the slice to the other, that means the angle of the slice is 1 radian. It's just  
another way of saying how big the angle is, like inches and centimeters are  
different ways of measuring length.

So, the `angle` tells you how much to `turn` or `how big the corner` is, and `radians`  
are a way of `measuring that angle`.

Explain like I am 5 when I would **`need to use radian`**
***

Alright, let's imagine a situation where you might need to use radians.

Imagine you're playing with a toy car on a big circular rug. You want your car  
to drive along the edge of the rug, but you want to know how far it needs to  
turn to go from one point to another along the edge.

When talking about turning or going around circles, we use radians to measure  
the turn. 

**`Radians help us understand how much of the circle we're covering`**.  

It's like making a cake and saying, "I want 1/4 of the cake," but instead, we  
say we want to turn 1/4 of a full circle.

So, when you're playing with your toy car on a circular rug, you use radians to  
help you measure how far it needs to turn to go from one point to another along  
the edge of the rug.

Explain the performance and memory difference between : 
- `directly` drawing polygon and `rotating the points` 
- `rasterizing` to a surface and `rotating the surface`
***

Both methods of rotating a polygon in Pygame - rotating the points and redrawing  
the polygon vs. drawing the polygon to a surface and rotating it - have their  
advantages and disadvantages in terms of performance and memory usage. Let's compare them:

1. **Rotating points and redrawing the polygon:**

   *Performance:*
   
   This method requires calculating the new positions of each vertex for every  
   frame. The calculations involve trigonometric functions, which can be  
   **`computationally expensive`**, especially for polygons with a **`large number`** of  
   **`vertice`**.

   *Memory:*
   
   This method typically uses less memory because you're only storing the  
   original polygon points and the calculated rotated points. You don't need to  
   create a separate surface for the polygon or store additional information.

2. **Drawing the polygon to a surface and rotating the surface:**

   *Performance:*
   
   This method may offer better performance for polygons with a large number of  
   vertices since it offloads the rotation task to the Pygame library, which  
   uses optimized C code and hardware acceleration (if available).  
   However, the performance can still be affected by the size of the surface  
   being rotated, as it involves **`copying`** and transforming **`pixel data`**.

   *Memory:*
   
   This method consumes more memory because you need to create a separate  
   surface for the polygon, and additional memory is used when creating a new  
   surface for the rotated version. The memory usage can increase with the size  
   of the surface, especially when using transparent surfaces with the  
   `pygame.SRCALPHA` flag.

In summary, rotating points and redrawing the polygon can be more  
computationally expensive, especially for polygons with many vertices.  
However, it uses less memory. Drawing the polygon to a surface and rotating the  
surface may offer better performance in some cases but consumes more memory due  
to the need for additional surfaces.

The best method depends on your specific use case and the performance and  
memory requirements of your application. For simple applications with a small  
number of polygons, either method should be sufficient. For more complex  
applications, you may need to test both methods to determine which one offers  
the best balance between performance and memory usage.

Explain **`type checking`** in python
***

Some key points:

1. Type hints only `optionally enforce types` - you need a separate type  
checking tool. But they `clearly document the expected types` for your function.

2. You can specify types for all function arguments and the return value.

3. Use List[type] to denote a list of elements of type type.

4. Use Tuple[type1, type2] to denote a tuple with two elements of types type1 and type2.

5. You can use int, str, float, bool, etc. to specify basic types. And List,  
Tuple, Dict to specify compound types.

6. Type hints `do not affect runtime behavior` - they only provide `static`  
type checks.

7. You can use a tool like `mypy` to actually `run the type checks and enforce`  
the hints. 

8. Specifying strict type hints makes your code more robust, documented and  
catch bugs early. 

You can use type checking to ensure that a function only accepts a list of  
points (x, y coordinates). You can do it like this:

```python
from typing import List, Tuple

def function_name(points: List[Tuple[int, int]]) -> None:
  """Only accepts a list of (x, y) point tuples"""
  # Function logic here 
```

This uses Type Hints to specify that the points argument must be a List of  
Tuples, where each Tuple has two int elements. 
So these would be valid calls:

```python
function_name([(1, 2), (3, 4)]) 
function_name([[1, 2], [3, 4]])  # Also ok, treats each sublist as a point
```

But these would throw a TypeError:

```python
function_name((1, 2, 3))  # Not a list 
function_name([(1, '2')])  # Not ints 
function_name(['string', 'list'])  # Not tuples
```

Explain **`type checking`** in python as if I am 5
***

Imagine you have a toy box with different sections for different toys:  

- cars
- dolls 
- and building blocks...etc 

Your mom tells you to put the toys into the right sections to keep the toy box  
organized.

In Python, the toys are like "values" or "data," and the sections in the toy box  
are like "types" (like numbers, words, or lists). Type checking is like making  
sure you put the toys into the correct sections.

When you write a program in Python, you use different types of data, like  
numbers, words, or lists. Type checking helps make sure you use the right type  
of data in your program.

For example, if you want to add two numbers together, you need to make sure they  
are both numbers, not a number and a word. Type checking helps you do that.

Python has two ways to do type checking:  

1. **Dynamic type checking**: Python automatically checks the types of the data  
when you run the program, just like your mom checks the toys in the box when  
you're done playing. If something is wrong, Python will tell you with an error  
message.

2. **Static type checking**: You can add "hints" to your program to tell Python  
what type of data you expect to use, just like putting labels on the toy box  
sections. This way, a tool like `mypy` or an editor like Visual Studio Code can  
check the types before you run the program and help you find any mistakes.

Type checking makes your program safer and easier to understand, just like  
keeping your toy box organized makes it easier to find and play with your toys.

Explain python **`set operations`** as if I’m 5
***

Imagine you have a box of different toy shapes:  
- triangles
- circles
- squares  

Now, let's say you have another box with shapes like circles, squares, and **`stars`**  
In Python, these boxes are like sets, and the shapes inside them are like the  
elements of the sets.

Now, let's learn some Python set operations using these toy shapes:

1. **Union (|)**: If you take all the shapes from both boxes and put them  
**`together`**, you will have a new box with all the shapes: 
- triangles 
- circles 
- squares 
- and **`stars`** 

This new box represents the union of the two sets.  

   ```python
   box1 = {"triangle", "circle", "square"}
   box2 = {"circle", "square", "star"}
   all_shapes = box1 | box2  # {"triangle", "circle", "square", "star"}
   ```

2. **Intersection (&)**: If you look for shapes that are in both box1 and box2,  
you will find circles and squares. This is the intersection of the two sets.

   ```python
   common_shapes = box1 & box2  # {"circle", "square"}
   ```

3. **Difference (-)**: If you want to find the shapes that are only in box1  
(and not in box2), you will have triangles. This is the difference between  
the two sets.

   ```python
   unique_shapes_box1 = box1 - box2  # {"triangle"}
   ```

4. **Symmetric Difference (^)**: If you want to find the shapes that are unique  
to each box (not in both), you will have triangles and stars. This is the  
symmetric difference between the two sets.

   ```python
   unique_shapes_both = box1 ^ box2  # {"triangle", "star"}
   ```

5. **issubset()**: If you want to check if all shapes in one box are also in  
another box, you can use `issubset()`. For example, if you have a small box with  
only circles and squares, you can check if all the shapes in the small box are  
also in box1.

   ```python
   small_box = {"circle", "square"}
   is_small_box_in_box1 = small_box.issubset(box1)  # True
   ```

These are the basic Python set operations explained using toy shapes. Just like  
you can play with toy shapes in the boxes, you can use set operations in Python  
to work with sets of elements.

Explain `*args` and `**kwargs`
***

Here's a brief explanation of *args and **kwargs:

- `*args`: This syntax allows you to pass a variable number of non-keyword  
**`positional`** arguments to a function. It collects all the positional arguments  
into a tuple. You can use any name instead of `args`, but it's a convention to  
use `args`.

- `**kwargs`: This syntax allows you to pass a variable number of **`keyword`**  
arguments to a function. It collects all the keyword arguments into a  
dictionary. Like with `args`, you can use any name instead of `kwargs`,  
but it's a convention to use `kwargs`.

Using *args and **kwargs in the `update` method of the `SystemABC` class allows  
for greater flexibility when defining subclasses that inherit from this abstract  
base class. By including *args and **kwargs,

Code review this **`naive`** implementation and explain refactoring options
***    

```python
def get_entities_with_all_components(self, component_types:List[Type]) -> List[Entity]:
    entities = []
    for entity_id, components in self.components.items():
        has_all_components = True
        for type in component_types:
            if type not in components:
                has_all_components = False
                break
        if has_all_components:
            entities.append(Entity(entity_id))
    return entities
```

`get_entities_with_all_components` - `GPT4` suggestion
***

Here's a reviewed and refactored version of your `get_entities_with_all_components`  
method:

```python
from typing import List, Type

def get_entities_with_all_components(self, component_types: List[Type]) -> List[Entity]:
    entities = []
    for entity_id, components in self.components.items():
        if all(component_type in components for component_type in component_types):
            entities.append(Entity(entity_id))
    return entities
```

Here are the changes I made:

1. Removed the `has_all_components` flag and used the `all()` built-in function  
instead. This function returns `True` if all elements of the iterable are true  
(or if the iterable is empty). By using a list comprehension inside `all()`,  
we can check if all the required component types are present in the `components`  
dictionary for the given entity. This simplifies the code and makes it more  
Pythonic.

`get_entities_with_all_components` - `Claude` suggestion
***

```python
def get_entities_with_all_components(self, component_types:List[Type]) -> List[Entity]:
    entities = []
    # @audit 💨 : Convert the list of component types to a set for faster lookup
    # - in O(1) time instead of O(n) time where n is # of component types
    # - order of component types passed in does not matter, any permutation  
    # will give the same result
    component_set = set(component_types)
    for entity_id, components in self.components.items():
        if component_set.issubset(set(components.keys())):
            entities.append(Entity(entity_id))
    return entities
```

**GPT4 comments**

This refactored version of the `get_entities_with_all_components` method  
introduces a few changes:

1. It converts the `component_types` list to a `component_set` set for faster  
lookup. Sets offer O(1) average time complexity for testing membership compared  
to lists, which have O(n) time complexity where n is the number of component  
types.

2. It uses the `issubset()` method to check if all component types are present  
in the `components` dictionary for the given entity. This method checks if every  
element in the `component_set` is in the specified iterable (in this case, a set  
of the component keys).

**Tradeoffs:**

1. **Performance**: The refactored version should have better performance in  
cases where the number of component types is large. This is because set  
membership tests are faster than list membership tests.

2. **Memory**: The refactored version uses a little more memory since it creates  
a set from the input list and another set from the component keys. This memory  
overhead should be negligible in most cases.

3. **Readability**: The refactored version is more concise and takes advantage  
of Python's built-in set operations, making it more Pythonic. However, it might  
be slightly less intuitive for developers who are not familiar with set  
operations.

**Functional differences:**

There are no significant functional differences between the two implementations.  
Both versions return a list of entities that have all the specified component  
types. The order of the component types passed to the function does not affect  
the result in either version.

Overall, the refactored version is a good improvement, especially if the `number`  
of `component types becomes large`. The performance gain from using sets for  
membership tests and the concise code using set operations make this version  
preferable in most cases.
